In [1]:
import config
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import cv2

In [3]:
from break_captcha import BreakCaptcha

In [4]:
options = webdriver.ChromeOptions()
#options.add_argument("--headless")
driver = webdriver.Chrome(
        config.CHROMEDRIVER,
        options=options
        )


C:\Users\Usuário\AppData\Local\Temp\ipykernel_10728\1766906669.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


In [19]:
url = 'http://pergamum.museu-goeldi.br:8080/pergamumweb/home_geral/login.jsp'
driver.get(url)

In [20]:
captcha = driver.find_element(By.ID, 'imgCaptcha')
captcha.screenshot('test.png')

True

In [21]:
resposta = BreakCaptcha('test.png')
resposta

1/1 [==============================] - 0s 73ms/step


'5MNF4'

In [6]:
username = driver.find_element(By.ID, 'id_txt_login')
username.send_keys('teste')


In [7]:
pasword = driver.find_element(By.ID, 'id_txt_senha')
pasword.send_keys('1111')

In [12]:
txtCaptcha = driver.find_element(By.ID, 'txtCaptcha')
txtCaptcha.send_keys(r)


In [13]:
submit = driver.find_element(By.ID, 'id_btn_ok')
submit.click()


In [14]:
driver.quit()

In [18]:
def Login(driver):
    driver.get(url)
    captcha = driver.find_element(By.ID, 'imgCaptcha')
    captcha.screenshot('test.png')
    r = BreakCaptcha('test.png')
    username = driver.find_element(By.ID, 'id_txt_login')
    username.send_keys('teste')
    pasword = driver.find_element(By.ID, 'id_txt_senha')
    pasword.send_keys('1111')
    txtCaptcha = driver.find_element(By.ID, 'txtCaptcha')
    txtCaptcha.send_keys(r)
    submit = driver.find_element(By.ID, 'id_btn_ok')
    submit.click()
    

In [19]:
Login(driver)

1/1 [==============================] - 0s 150ms/step
